In [1]:
import struct
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import pywt
import six

In [2]:
def get_file(path):
    global filelist
    filelisttemp = os.listdir(path)
    for filename in filelisttemp:
        if os.path.isdir(path + "/" + filename):
            get_file(path + "/" + filename)
        else:
            filelist.append(path + "/" + filename)

In [3]:
def save_file(writer):
    interval = 5
    dt = 0.0000001 * interval
    fs = 10000000 / interval
    start = 250000
    end = 450000
    fig_size = 40

    global filelist
    for path in filelist:
        with open(path, "rb") as fb:
            data = fb.read()

        ch1ch2 = struct.unpack("<"+str(int(len(data)/2))+"H", data)
        ch1ch2 = np.array(ch1ch2)
        ch1ch2 = (ch1ch2-8192)*2.5/8192

        datay1 = ch1ch2[::2]
        #datay2 = ch1ch2[1::2]
        
        data1 = datay1[start:end:interval]
        #data2 = datay2[start:end:interval]

        wavelet = 'morl'
        c = pywt.central_frequency(wavelet)
        fa = np.arange(400000, 20000 - 1, -20000)
        scales = np.array(float(c)) * fs / np.array(fa)

        [cfs1,frequencies1] = pywt.cwt(data1,scales,wavelet,dt)
        #[cfs2,frequencies2] = pywt.cwt(data2,scales,wavelet,dt)
        power1 = abs(cfs1)
        #power2 = (abs(cfs2))

        length_now = len(power1[0])
        power1 = np.reshape(power1,(len(power1),fig_size,int(length_now/fig_size)))
        #power2 = np.reshape(power2,(len(power2),fig_size,int(length_now/fig_size)))
        power1 = np.log10(np.mean(power1,axis=2))
        #power2 = np.log10(np.mean(power2,axis=2))

        mx = power1.max()
        mn = power1.min()
        power1 = (power1-mn) / (mx-mn) * 255.0
        power1 = power1.astype(np.uint8)
        power1 = power1.tobytes()

        #mx = power2.max()
        #mn = power2.min()
        #power2 = (power2-mn) / (mx-mn) * 255.0
        #power2 = power2.astype(np.uint8)
        #power2 = power2.tobytes()

        example = tf.train.Example(features=tf.train.Features(feature={'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[power1]))}))
        writer.write(example.SerializeToString())
        #example = tf.train.Example(features=tf.train.Features(feature={'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[power2]))}))
        #writer.write(example.SerializeToString())

In [4]:
filelist = []

recordname = "/home/adoge/AE-location/input_data/sand"
get_file(recordname)
filelist = np.array(filelist)
np.random.shuffle(filelist)

writer = tf.python_io.TFRecordWriter("AE_input_sand_ch1.tfrecord")
save_file(writer)

writer.close()